In [2]:
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import TimeoutException
import pandas as pd

import numpy as np
import joblib
import time

In [3]:
#addresX='sp/sao-paulo/zona-oeste/rio-pequeno/'

#url = f'https://www.vivareal.com.br/venda/{addresX}/apartamento_residencial/'
#driver = webdriver.Chrome(r"C:\Users\a92550\Downloads\chromedriver.exe")
#driver.get(url)
#page = driver.page_source
        
#soup = BeautifulSoup(page,features="html.parser")
#propriedades = soup.find_all('article',{'class': 'property-card__container js-property-card'})

In [4]:
#p = propriedades[1]

In [5]:
def extract_page(addres,npages,status='todos'):

    data = {'ID':[],'area':[],'preço':[],'condominio':[],'quartos':[],'vagas':[],'status':[],'detalhes':[],'endereço':[],'link':[]}
    
    if (status == 'todos'):
        url = f'https://www.vivareal.com.br/venda/{addres}/apartamento_residencial/'
    elif (status == 'novos'):
        url = f'https://www.vivareal.com.br/imoveis-lancamento/{addres}/apartamento_residencial/#tipos-lancamento=apartamento_residencial'
    
    driver = webdriver.Chrome(r"C:\Users\a92550\Downloads\chromedriver.exe")
    driver.get(url)
    
    for n in range(0,npages):
               
        page = driver.page_source
        
        soup = BeautifulSoup(page,features="html.parser")
        propriedades = soup.find_all('article',{'class': 'property-card__container js-property-card'})
        
        for p in propriedades:
                        
            try: data['ID'].append(p.a['href'][-11:-1])
            except:data['ID'].append('') 
                                
            try: data['area'].append(p.find('span',{'class': 'property-card__detail-value js-property-card-value property-card__detail-area js-property-card-detail-area'}).text)
            except:data['area'].append('')      
            
            try: data['preço'].append(p.find('div',{'class': 'property-card__price js-property-card-prices js-property-card__price-small'}).text)
            except:data['preço'].append('')
                
            try: data['condominio'].append(p.find('div',{'class':'property-card__price-details--condo'}).strong.text)
            except:data['condominio'].append('')
        
            try: data['quartos'].append(p.find('li',{'class': 'property-card__detail-item property-card__detail-room js-property-detail-rooms'}).find('span').text)
            except:data['quartos'].append('')
        
            try: data['vagas'].append(p.find('li',{'class': 'property-card__detail-item property-card__detail-garage js-property-detail-garages'}).find('span').text)
            except:data['vagas'].append('')
            
            try: data['endereço'].append(p.find('span',{'class': 'property-card__address'}).text)
            except:data['endereço'].append('')
            
            try: data['status'].append(p.find('div',{'class':'property-card__construction-status js-construction-status'}).text)
            except: data['status'].append('usado')
            
            try: 
                detalhes = []
                for a in p.find('ul',{'class': 'property-card__amenities'}).find_all('li'):
                    detalhes.append(a.text)
            
                data['detalhes'].append('|'.join(detalhes))
            except:data['detalhes'].append('')
            
            try: data['link'].append(p.a['href'])
            except:data['link'].append('')
            
        try:
            driver.execute_script("arguments[0].scrollIntoView();", WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="js-site-main"]/div[2]/div[1]/section/div[2]/div[2]/div/ul/li[9]/a'))))
            driver.find_element_by_xpath('//*[@id="js-site-main"]/div[2]/div[1]/section/div[2]/div[2]/div/ul/li[9]/a').click()
            time.sleep(5)
        except:
            print("Last page reached")
            break
            
    driver.quit()
        
    return pd.DataFrame(data)



In [6]:
#jaguare = extract_page(addres='sao-paulo',npages=100)
#butanta = extract_page(addres='sp/sao-paulo/zona-oeste/butanta/',npages=100)
#rio_pequeno = extract_page(addres='sp/sao-paulo/zona-oeste/rio-pequeno/',npages=100)
#barra_funda = extract_page(addres='sp/sao-paulo/zona-oeste/barra-funda/',npages=100)

In [7]:
#belenzinho = extract_page(addres='sp/sao-paulo/zona-leste/belenzinho/',npages=100)
#bras = extract_page(addres='sp/sao-paulo/zona-leste/bras/',npages=100)
#tatuape = extract_page(addres='sp/sao-paulo/zona-leste/tatuape/',npages=100)
#carrao = extract_page(addres='sp/sao-paulo/zona-leste/carrao/',npages=100)

In [8]:
#vila_mariana = extract_page(addres='sp/sao-paulo/zona-sul/vila-mariana/',npages=100)
#vila_clementino = extract_page(addres='/venda/sp/sao-paulo/zona-sul/vila-clementino/',npages=100)
#saude = extract_page(addres='sp/sao-paulo/zona-sul/saude/',npages=100)

In [9]:
#liberdade = extract_page(addres='sp/sao-paulo/centro/liberdade/',npages=100)
#republica = extract_page(addres='sp/sao-paulo/centro/republica/',npages=100)
#se = extract_page(addres='sp/sao-paulo/centro/se/',npages=100)

In [10]:
lanç_saopaulo = extract_page(addres='sao-paulo',npages=100,status='novos')
lanç_campinas = extract_page(addres='campinas',npages=100,status='novos')
lanç_rio = extract_page(addres='rio-de-janeiro',npages=100,status='novos')
lanç_osasco = extract_page(addres='osasco',npages=100,status='novos')
lanç_BH = extract_page(addres='belo-horizonte',npages=100,status='novos')
lanç_salvador = extract_page(addres='salvador',npages=100,status='novos')

Last page reached
Last page reached
Last page reached
Last page reached
Last page reached
Last page reached


In [41]:
df = pd.concat([lanç_saopaulo,lanç_campinas,lanç_osasco,lanç_rio,lanç_BH,lanç_salvador])
df

,ID,area,preço,condominio,quartos,vagas,status,detalhes,endereço,link
0,2525798998,34-115,Sob Consulta,,1-3,1-2,Na planta,,"Avenida Onze de Junho, 222 - Vila Clementino, ...",/imoveis-lancamento/onze-vila-clementino-25257...
1,2533789894,32-37,Sob Consulta,,1-2,--,Na planta,,"Rua Pontes de Morais, 610 - Jardim da Pedreira...",/imoveis-lancamento/show-zona-sul-2533789894/
2,2533791093,37-44,Sob Consulta,,1-2,--,Na planta,,"Rua Anhangüera, 396 - Barra Funda, São Paulo - SP",/imoveis-lancamento/staz-2533791093/
3,2500254991,30-65,R$ 388.000,,1-2,1,Em construção,,"Rua Vieira de Morais, 79 - Campo Belo, São Pau...",/imoveis-lancamento/neohouse-campo-belo-250025...
4,2523639959,37-39,R$ 192.440,,1-2,--,Na planta,,"Avenida Sapopemba, 7595 - Vila Virginia, São P...",/imoveis-lancamento/grow-sapopemba-2523639959/
...,...,...,...,...,...,...,...,...,...,...
20,2519879425,110,Sob Consulta,,3,2,Pronto para morar,,"Rua Embira, 149 - Patamares, Salvador - BA",/imoveis-lancamento/platno-greenville-2519879425/
21,2514711784,134-180,Sob Consulta,,3-4,2,Pronto para morar,,"Rua Salgueiro, 379 - Patamares, Salvador - BA",/imoveis-lancamento/ludco-greenville-2514711784/
22,2519878164,53-71,Sob Consulta,,2-3,1,Pronto para morar,,"3ª Travessa Paripiranga, 760 - Bairro da Paz, ...",/imoveis-lancamento/flex-piata-2519878164/
23,2538882822,128,Sob Consulta,,3-4,3,Na planta,,"Avenida Alphaville, - - Alphaville I, Salvador...",/imoveis-lancamento/haus-residence-2538882822/


In [45]:
df2 = df['area'].str.split('-',expand=True)
df3 = df['quartos'].str.split('-',expand=True)
df4 = df['vagas'].str.split('-',expand=True)

In [46]:
df2

,0,1
0,34,115
1,32,37
2,37,44
3,30,65
4,37,39
...,...,...
20,110,None
21,134,180
22,53,71
23,128,None


In [30]:
#Transformação do DataFrame

df['preço'] = df['preço'].str.replace(r"[^0-9]+",'')
df['condominio'] = df['condominio'].str.replace(r"[^0-9]+",'')

df['max quartos'] = df['max quartos'].str.replace(r"[^0-9]+",'')
df['min quartos'] = df['min quartos'].str.replace(r"[^0-9]+",'')

df['max area'] = df['max area'].str.replace(r"[^0-9]+",'')
df['min area'] = df['min area'].str.replace(r"[^0-9]+",'')

df['max vagas'] = df['max vagas'].str.replace(r"[^0-9]+",'')
df['min vagas'] = df['min vagas'].str.replace(r"[^0-9]+",'')

df['quarto'] = (df['min quartos'] + df['max quartos'])/2
df['area'] = (df['min area'] + df['max area'])/2
df['vagas'] = (df['min vagas'] + df['max vagas'])/2





KeyError: 'max quartos'

In [17]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1638 entries, 0 to 24
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID          1638 non-null   object
 1   area        1638 non-null   object
 2   preço       1638 non-null   object
 3   condominio  1638 non-null   object
 4   quartos     1638 non-null   object
 5   vagas       1638 non-null   object
 6   status      1638 non-null   object
 7   detalhes    1638 non-null   object
 8   endereço    1638 non-null   object
 9   link        1638 non-null   object
dtypes: object(10)
memory usage: 140.8+ KB


,ID,area,preço,condominio,quartos,vagas,status,detalhes,endereço,link
count,1638,1638,1638,1638,1638,1638,1638,1638,1638,1638
unique,1581,327,970,1,7,11,3,1,1308,1581
top,2513794676,43,,,2,,Na planta,,"Rua Real Grandeza, 96 - Botafogo, Rio de Janei...",/imoveis-lancamento/veritas-2500859473/
freq,2,38,414,1638,562,552,747,1638,5,2


In [18]:
df = df.drop_duplicates()

In [19]:
dummies = df['detalhes'].str.get_dummies('|')
df = pd.concat([df,dummies],axis=1).drop(['...','detalhes'],axis=1)


KeyError: "['...'] not found in axis"

In [ ]:
#exportar
df.to_excel('imoveis_novos.xlsx')